<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/vectordb_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 11.7 MB/s et

In [2]:
documents = [
    "The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan.",
    "Python is a popular programming language for AI and data science.",
    "The solar system has eight planets revolving around the sun.",
    "The cat sat on the mat.",
]

In [3]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
embeddings = embedding_model.encode(documents).tolist()

In [5]:
import chromadb
from chromadb.config import Settings

In [6]:
client = chromadb.PersistentClient(path="./chroma_db")

In [7]:
collection = client.get_or_create_collection(name="rag_demo")

In [8]:
for i, (doc, emb) in enumerate(zip(documents, embeddings)):
    collection.add(
        documents=[doc],
        embeddings=[emb],
        ids=[str(i)]
    )

In [9]:
query = "Where is the Taj Mahal?"
query_embedding = embedding_model.encode(query).tolist()

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=1
)

print("\nQuery Results:")


Query Results:


In [10]:
for i, result in enumerate(results['documents'][0]):
    print(f"{i+1}. {result}")

count = collection.count()
print(f"Number of items in the collection: {count}")

include=['embeddings', 'documents', 'metadatas'] # Specify what to include in the results

retrieved_items = collection.get(include=include)

print("\nRetrieved Items:")

1. The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan.
Number of items in the collection: 4

Retrieved Items:


In [11]:
retrieved_items = collection.get(include=include)

print("\nRetrieved Items:")
for i, item_id in enumerate(retrieved_items['ids']):
    print(f"Item ID: {item_id}")
    print(f"Document: {retrieved_items['documents'][i]}")
    # print(f"Embedding: {retrieved_items['embeddings'][i]}") # Uncomment to see embeddings
    # print(f"Metadata: {retrieved_items['metadatas'][i]}") # Uncomment to see metadatas
    print("-" * 20)


Retrieved Items:
Item ID: 0
Document: The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan.
--------------------
Item ID: 1
Document: Python is a popular programming language for AI and data science.
--------------------
Item ID: 2
Document: The solar system has eight planets revolving around the sun.
--------------------
Item ID: 3
Document: The cat sat on the mat.
--------------------


In [12]:
from google.colab import files

uploaded_files = files.upload()

Saving Apriori - Google Docs.pdf to Apriori - Google Docs.pdf


In [13]:
!pip install -q PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.3 MB/s eta 0:00:00


In [14]:
import fitz  # PyMuPDF

extracted_texts = {}
for filename, content in uploaded_files.items():
    try:
        # Save the uploaded file temporarily to process it with PyMuPDF
        with open(filename, 'wb') as f:
            f.write(content)

        # Open the PDF file
        doc = fitz.open(filename)
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text()

        extracted_texts[filename] = text
        print(f"Processed file: {filename}")

    except Exception as e:
        print(f"Error processing file {filename}: {e}")

print("\nExtracted Texts:")
for filename, text in extracted_texts.items():
    print(f"--- {filename} ---")
    print(text[:200] + "..." if len(text) > 200 else text) # Print first 200 characters
    print("-" * 20)

Processed file: Apriori - Google Docs.pdf

Extracted Texts:
--- Apriori - Google Docs.pdf ---
​How the Apriori Algorithm Works​
​Step 1: Set a minimum support threshold​
​●​ ​Eliminate itemsets whose support is less than the threshold.​
​Step 2: Find all frequent itemsets​
​●​ ​Generate candid...
--------------------


In [16]:
new_embeddings = {}
for filename, text in extracted_texts.items():
    try:
        embedding = embedding_model.encode(text).tolist()
        new_embeddings[filename] = embedding
        print(f"Generated embedding for: {filename}")
    except Exception as e:
        print(f"Error generating embedding for {filename}: {e}")

print("\nGenerated Embeddings (first 5 elements):")
for filename, embedding in new_embeddings.items():
    print(f"--- {filename} ---")
    print(embedding[:5])
    print("-" * 20)

try:
    for filename, text in extracted_texts.items():
        embedding = new_embeddings.get(filename)
        if embedding:
            collection.add(
                documents=[text],
                embeddings=[embedding],
                ids=[filename]  # Using filename as the ID
            )
            print(f"Added document '{filename}' to ChromaDB.")
        else:
            print(f"Skipping '{filename}' as embedding was not generated.")

    print("\nSuccessfully added new documents to ChromaDB.")

except Exception as e:
    print(f"Error adding documents to ChromaDB: {e}")

print(f"Number of items in the collection: {count}")

Generated embedding for: Apriori - Google Docs.pdf

Generated Embeddings (first 5 elements):
--- Apriori - Google Docs.pdf ---
[-0.04254218190908432, 0.03461726754903793, -0.027040058746933937, -0.0008772346191108227, 0.029651256278157234]
--------------------
Added document 'Apriori - Google Docs.pdf' to ChromaDB.

Successfully added new documents to ChromaDB.
Number of items in the collection: 4


In [17]:
retrieved_items = collection.get(
    ids=["0", "1", "Resume2025V2.pdf"], # Example: retrieve items with ids "0", "1" and the new document's filename
    include=['embeddings', 'documents', 'metadatas'] # Specify what to include in the results
)

print("\nRetrieved Items:")
for i, item_id in enumerate(retrieved_items['ids']):
    print(f"Item ID: {item_id}")
    print(f"Document: {retrieved_items['documents'][i][:200]}...") # Print first 200 characters of the document
    # print(f"Embedding: {retrieved_items['embeddings'][i]}") # Uncomment to see embeddings
    # print(f"Metadata: {retrieved_items['metadatas'][i]}") # Uncomment to see metadatas
    print("-" * 20)


Retrieved Items:
Item ID: 0
Document: The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan....
--------------------
Item ID: 1
Document: Python is a popular programming language for AI and data science....
--------------------
